### Reference Kernel
[Kernel 1](https://www.kaggle.com/nikunjm88/creating-additional-features)<br>
[Kernel 2](https://www.kaggle.com/jamesdhope/zillow-ensemble-of-regressors-0-065)<br>

### Import Data

In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from subprocess import check_output
print(check_output(["ls", "./dataset"]).decode("utf8"))


properties_2016.csv
properties_2016.csv.zip
train_2016_v2.csv
train_2016_v2.csv.zip
zillow_data_dictionary.xlsx
zillow_data_dictionary.xlsx.zip
~$zillow_data_dictionary.xlsx



In [2]:
sns.set(style='white', context='notebook', palette='deep')

properties = pd.read_csv('./dataset/properties_2016.csv')
train = pd.read_csv("./dataset/train_2016_v2.csv")
print(properties.shape)
print(train.shape)

(2985217, 58)
(90275, 3)


In [3]:
properties.head(10)

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN
5,10898347,NaN,NaN,NaN,0.0,0.0,4.0,7.0,NaN,NaN,...,1.0,NaN,176383.0,283315.0,2015.0,106932.0,3661.28,NaN,NaN,NaN
6,10933547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,397945.0,554573.0,2015.0,156628.0,6773.34,NaN,NaN,NaN
7,10940747,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,1.0,NaN,101998.0,688486.0,2015.0,586488.0,7857.84,NaN,NaN,NaN
8,10954547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
9,10976347,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,218440.0,261201.0,2015.0,42761.0,4054.76,NaN,NaN,NaN


In [4]:
ParcelId = properties['parcelid']

### Feature Engineering


In [5]:
#print(properties.isnull().sum())

In [6]:
#print(properties.info())

- basementsqft: Finished living area below or partially below ground level
- fquarefinishedseet12: Finished living area
- finishedsquarefeet13: Perimeter  living area
- finishedsquarefeet15: Total area
- finishedsquarefeet50: Size of the finished living area on the first (entry) floor of the home

In [7]:
#properties['basementsqft'].head(10)
dummy = properties.loc[~properties['basementsqft'].isnull()]
print(dummy.shape)
dummy = dummy.loc[:,['basementsqft','fquarefinishedseet12','finishedsquarefeet13','finishedsquarefeet15','finishedsquarefeet50']]
dummy.head(10)

(1628, 58)


,basementsqft,fquarefinishedseet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50
377,216.0,NaN,NaN,NaN,1348.0
379,555.0,NaN,NaN,NaN,1245.0
383,224.0,NaN,NaN,NaN,1314.0
486,782.0,NaN,NaN,NaN,2419.0
859,651.0,NaN,NaN,NaN,1371.0
861,516.0,NaN,NaN,NaN,1754.0
1037,600.0,NaN,NaN,NaN,600.0
1038,732.0,NaN,NaN,NaN,1204.0
15100,1145.0,NaN,NaN,NaN,1701.0
15127,220.0,NaN,NaN,NaN,1920.0


奇怪，連total area(finishedsquarefeet15)的資料都沒有，<br>
finishedsquarefeet15有2794419(93.6%)筆是null，<br>
basementsqft有2983589(99.9%)筆是null

In [8]:
properties['has_basement'] = properties['basementsqft'].apply(lambda x: 0 if np.isnan(x) else 1).astype(np.float64)
#dummy = properties.loc[~properties['has_basement'].isnull()]
#dummy.loc[:,['basementsqft', 'has_basement']].head(5)

In [9]:
#dummy.loc[:,['basementsqft', 'has_basement']].dropna().head(5)

- hashottuborspa(2916203 isnull, object): Does the home have a hot tub or spa

In [10]:
dummy = properties.loc[:,['hashottuborspa']].dropna()
print(dummy.shape)
dummy = dummy.loc[dummy['hashottuborspa']==True]
print(dummy.shape)
dummy.head(5)

(69014, 1)
(69014, 1)


,hashottuborspa
1340,True
1341,True
1497,True
1691,True
1694,True


由上可知，hashottuborspa所有非NaN的皆是True

In [11]:
properties['hashottuborspa'] = properties['hashottuborspa'].apply(lambda x: 0 if np.isnan(x) or x==False else 1).astype(np.int64)

- poolcnt(2467683 isnull, float64):  Number of pools on the lot (if any)

In [12]:
#properties.loc[properties['poolcnt']==0].head(5) # the resoult is 0 rows, so no poolcnt is 0
properties['has_pool'] = properties['poolcnt'].apply(lambda x: 0 if np.isnan(x) or x==0 else 1).astype(np.int64)

- airconditioningtypeid(2173698 isnull, float64): Type of cooling system present in the home (if any)

In [13]:
properties['has_airconditioning'] = properties['airconditioningtypeid'].apply(lambda x: 0 if np.isnan(x) else 1).astype(np.int64)

- yardbuildingsqft17(2904862 isnull, float64): Patio in yard
- yardbuildingsqft26(2982570 isnull, float64): Storage shed/building in yard

In [14]:
properties['yardbuildingsqft17'] = properties['yardbuildingsqft17'].apply(lambda x: 0 if np.isnan(x) else x).astype(np.float64)
properties['yardbuildingsqft26'] = properties['yardbuildingsqft26'].apply(lambda x: 0 if np.isnan(x) else x).astype(np.float64)
properties['yard_building_square_feet'] = properties['yardbuildingsqft17'].astype(np.int64) + properties['yardbuildingsqft26'].astype(np.int64)